In [10]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

pd.options.display.max_rows = 50
%matplotlib inline

# Biopython Entrez
Provides code to access NCBI over the WWW.  

[Package info](https://biopython.org/DIST/docs/api/Bio.Entrez-module.html)

* `api_key` Personal API key from NCBI. If not set, only 3 queries per second are allowed. 10 queries per seconds otherwise with a valid API key.

### 1. Basics

In [2]:
from Bio import Entrez

In [3]:
from Bio import SeqIO

In [4]:
handle = Entrez.efetch(db="nucleotide", id="NC_045512",
                       rettype="gb", retmode="text")
handle.readline().strip()

'LOCUS       NC_045512              29903 bp ss-RNA     linear   VRL 18-JUL-2020'

### 2. Search via `esearch`

In [6]:
handle = Entrez.esearch(db="nucleotide", retmax=10, term="SARS-CoV-2", idtype="acc")
record = Entrez.read(handle)
handle.close()
print(record)

{'Count': '12266', 'RetMax': '10', 'RetStart': '0', 'IdList': ['NC_045512.2', 'NM_021804.3', 'NM_001371415.1', 'MT773134.1', 'MT773133.1', 'MT772580.1', 'MT772579.1', 'MT772578.1', 'MT772577.1', 'MT772576.1'], 'TranslationSet': [{'From': 'SARS-CoV-2', 'To': '"Severe acute respiratory syndrome coronavirus 2"[Organism] OR SARS-CoV-2[All Fields]'}], 'TranslationStack': [{'Term': '"Severe acute respiratory syndrome coronavirus 2"[Organism]', 'Field': 'Organism', 'Count': '11145', 'Explode': 'Y'}, {'Term': 'SARS-CoV-2[All Fields]', 'Field': 'All Fields', 'Count': '12266', 'Explode': 'N'}, 'OR', 'GROUP'], 'QueryTranslation': '"Severe acute respiratory syndrome coronavirus 2"[Organism] OR SARS-CoV-2[All Fields]'}


### 3. Method From `coronaversing`
To fetch the latest data from NCBI using keywords and a genome size range.

In [7]:
# data folders
root_folder = "./data/genbank/"
viruses = ["coronaviridae"]

# file periodically removed to update with the latest data
cache_vrs_file = ".cache_vrs.pkl" 
cache_cds_file = ".cache_cds.pkl" 
cache_regions_file = ".cache_regions.pkl" 

hidden_columns = ["sequence", "file_path", "strain", "collection_date"]
hidden_cds_columns = ["translation", "file_path", "location"]
hidden_regions_columns = ["id", "rna", "protein", "path"]

base_folder = "../data/genbank/"
# Entrez.email = 'dna-toolbox@gmail.com'

size_and_date = "(\"25000\"[SLEN] : \"35000\"[SLEN])"
genbank_search_query = {"coronaviridae": ["(\"Coronaviridae\"[Organism]) AND " + size_and_date]}

def fetch_gb(path, gid):
    handle = Entrez.efetch(db="nucleotide", id=gid, rettype="gb")
    local_file = open(path + "/" + gid + ".gb", 'w')
    local_file.write(handle.read())
    handle.close()
    local_file.close()

genome_files = genbank_search_query
viruses = genbank_search_query.keys()

In [11]:
count = 0
for vrs in viruses:
    
    data_folder = base_folder + vrs
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    
    for keywords in genome_files[vrs]:
        handle = Entrez.esearch(db="nucleotide", retmax=5000, term=keywords, idtype="acc")
        record = Entrez.read(handle)
        ids = record["IdList"]
        print("[+] got", len(ids), "records")
        handle.close()

        for genbank_id in ids:
            file_name = data_folder + "/" + genbank_id + ".gb"
            count += 1
            if not os.path.exists(file_name):
                print("[+]  --- * fetching (", vrs, ")", file_name)
                downloaded = False
                tries = 0
                while not downloaded:
                    try:
                        fetch_gb(data_folder, genbank_id)
                        downloaded = True
                    except:
                        if tries == 5:
                            print("[+]  --- * couldn't get (", vrs, ")", genbank_id)
                            break
                        tries += 1
                        time.sleep(3)
                        pass

print("[+] Collected", count, " gb files")

[+] got 5000 records
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MN759311.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/NC_045512.2.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/NC_004718.3.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT773134.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT773133.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772580.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772579.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772578.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772577.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772576.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772575.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT77257

[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772485.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772484.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772483.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772482.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772481.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772480.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772479.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772478.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772477.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772476.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772475.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772474.1.gb
[+]  --- * fetch

[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT772085.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT766908.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT766907.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/LR860655.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/LR860654.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/LR860653.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/LR860652.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/LR860651.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/LR860650.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/LR860649.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/LR860648.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/LR860647.1.gb
[+]  --- * fetch

[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT764185.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT764184.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT764183.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT764182.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT764181.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT764180.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT764179.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT764178.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT764177.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT764176.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT764175.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT764174.1.gb
[+]  --- * fetch

[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT757080.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT757079.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT757078.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT757077.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT757076.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT757075.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT757074.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT757073.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT757072.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT757071.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT757070.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT757069.1.gb
[+]  --- * fetch

...

[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT631827.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT631826.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT631825.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT631824.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT631823.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT631822.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT631821.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT631820.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT631819.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT631818.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT631817.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT631816.1.gb
[+]  --- * fetch

[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628237.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628236.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628235.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628234.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628233.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628232.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628231.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628230.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628229.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628228.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628227.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628226.1.gb
[+]  --- * fetch

[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628137.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628136.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628135.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628134.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628133.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628132.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628131.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628130.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628129.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628128.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628127.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628126.1.gb
[+]  --- * fetch

[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT628037.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT627932.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT627931.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT627930.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT627929.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT627928.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT627927.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT627926.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT627925.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT627924.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT627923.1.gb
[+]  --- * fetching ( coronaviridae ) ../data/genbank/coronaviridae/MT627922.1.gb
[+]  --- * fetch

[+] Collected 5000  gb files


In [12]:
1


1